In [1]:
import pandas as pd 
import numpy as np 
from datetime import date
from optimisation_utiles import * 
df = pd.read_csv('ARP_Returns.csv')[:3026]
df_return = cal_return(df)
# My weekly change
df_weekly = df_return.resample('W-FRI').apply(lambda x: (1+x).prod()-1)

In [2]:
df_weekly = df_weekly[df_weekly.index>'2009-09-01']
df_weekly = df_weekly.drop(columns=['equity_momentum', 'equity_low_beta', 'equity_quality', 'equity_trend', 'fx_value', 'commodity_carry'])
df_return = df_return[df_return.index>'2009-09-01']
df_return = df_return.drop(columns=['equity_momentum', 'equity_low_beta', 'equity_quality', 'equity_trend', 'fx_value', 'commodity_carry'])


In [5]:
%%time
div_results = rolling_portfolio_optimisation(df_return=df_weekly, default_upper_bound=0.1, default_lower_bound=0, window_size=36, optimisation_freq=12, target='diversification_ratio', input_bounds=None, product='ARP')

2009-09-27 ---- 2012-09-28
2010-09-26 ---- 2013-09-29
2011-09-30 ---- 2014-09-28
2012-09-28 ---- 2015-09-27
2013-09-29 ---- 2016-09-30
2014-09-28 ---- 2017-09-29
2015-09-27 ---- 2018-09-28
2016-09-30 ---- 2019-09-29
2017-09-29 ---- 2020-09-27
CPU times: user 35.6 s, sys: 365 ms, total: 36 s
Wall time: 9.24 s


In [6]:
input_bounds_max_sharpe = Duncans_weights(df_weekly, div_results)

2012-09-28
2013-09-29
2014-09-28
2015-09-27
2016-09-30
2017-09-29
2018-09-28
2019-09-29
2020-09-27


In [7]:
# cap vol carry at 0.3
for date in input_bounds_max_sharpe:
    input_bounds_max_sharpe[date]['interest_rate_volatility_carry'] = [0, 0.03]
    input_bounds_max_sharpe[date]['commodity_volatility_carry'] = [0, 0.03]
    input_bounds_max_sharpe[date]['equity_volatility_carry'] = [0, 0.03]

In [48]:
sharpe_results = rolling_portfolio_optimisation(df_return=df_weekly, default_upper_bound=1, default_lower_bound=0, window_size=36, optimisation_freq=12, target='sharpe_ratio', input_bounds=input_bounds_max_sharpe)
df_component_weights_s, df_weights_s = generate_component_weights(df_weekly, sharpe_results)
df_component_weights_d, df_weights_d = generate_component_weights(df_weekly, div_results)

2009-09-27 ---- 2012-09-28
2010-09-26 ---- 2013-09-29
2011-09-30 ---- 2014-09-28
2012-09-28 ---- 2015-09-27
2013-09-29 ---- 2016-09-30
2014-09-28 ---- 2017-09-29
2015-09-27 ---- 2018-09-28
2016-09-30 ---- 2019-09-29
2017-09-29 ---- 2020-09-27


In [49]:
from PortfolioPerformance import * 
rebalance_results_s = Get_Portfolio_Index_Base_Case(df_return, df_component_weights_s, initial_notional=100)
rebalance_results_d = Get_Portfolio_Index_Base_Case(df_return, df_component_weights_d, initial_notional=100)

In [50]:
df_component_return_s = rebalance_results_s['PnL_df_net']
df_component_return_s = df_component_return_s[(df_component_return_s.index >= '2012-09-29')]
df_component_return_d = rebalance_results_d['PnL_df_net']
df_component_return_d = df_component_return_d[(df_component_return_d.index >= '2012-09-29')]

In [84]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [89]:
def save_results(rolling_result, rebalance_results, which='div'):
    # weights 
    df_weights = pd.DataFrame()
    df_risk_contrib = pd.DataFrame()
    df_risk_contrib_pct = pd.DataFrame()
    df_return_contrib = pd.DataFrame()

    df_cum_return = rebalance_results['portfolio_index']
    df_cum_return = df_cum_return[df_cum_return.index>='2012-09-20']

    df_component_return = rebalance_results['PnL_df_net']
    df_component_return = df_component_return[df_component_return.index>='2012-09-29']
    full_stats = get_stats(weights='-', df_return=df_component_return, realised_return=True, frequency='daily')

    df_full_stats = pd.concat([pd.DataFrame(full_stats['component_risk_contribution'], index=df_weekly.columns, columns=['component_risk_contri']).T,
                    pd.DataFrame(full_stats['component_risk_contribution_pct'], index=df_weekly.columns, columns=['component_risk_contri_pct']).T,
                    pd.DataFrame(full_stats['component_total_return_contribution'], index=df_weekly.columns, columns=['component_total_return_contri']).T])


    for date in rolling_result:
        df_weights = pd.concat([df_weights, pd.DataFrame(rolling_result[date]['weights'], index=[date])])
        df_risk_contrib = pd.concat([df_risk_contrib, pd.DataFrame(div_results[date]['component_risk_contribution'], index=df_weekly.columns, columns=[date]).T])
        df_risk_contrib_pct = pd.concat([df_risk_contrib_pct, pd.DataFrame(div_results[date]['component_risk_contribution_pct'], index=df_weekly.columns, columns=[date]).T])
        df_return_contrib = pd.concat([df_return_contrib, pd.DataFrame(div_results[date]['component_total_return_contribution'], columns=[date]).T])

    writer = pd.ExcelWriter(f'{which}.xlsx', engine='xlsxwriter')
    df_cum_return.to_excel(writer, sheet_name='performance')
    df_full_stats.to_excel(writer, sheet_name='overall_risk_stats')
    df_weights.to_excel(writer, sheet_name='weights')
    df_risk_contrib.to_excel(writer, sheet_name='risk_contribution')
    df_risk_contrib_pct.to_excel(writer, sheet_name='risk_contribution_pct')
    df_return_contrib.to_excel(writer, sheet_name='return_contribution')
    writer.save()

    return full_stats
    


In [94]:
full_stats_d = save_results(div_results, rebalance_results_d, which='div')
full_stats_d

{'return': 0.6780700799289189,
 'volatility': 0.9023947053611945,
 'sharpe_ratio': 0.7514118554779342,
 'diversification_ratio': 3.4709124533855973,
 'marginal_risk_contribution': array([0.09408912, 0.03301523, 0.06757587, 0.10056451, 0.02236913,
        0.07666518, 0.0414154 , 0.00413443, 0.0235958 , 0.0441377 ,
        0.08949875, 0.03216053, 0.03351474, 0.01741535, 0.08155662,
        0.03129719, 0.10938915]),
 'component_risk_contribution': array([0.09408912, 0.03301523, 0.06757587, 0.10056451, 0.02236913,
        0.07666518, 0.0414154 , 0.00413443, 0.0235958 , 0.0441377 ,
        0.08949875, 0.03216053, 0.03351474, 0.01741535, 0.08155662,
        0.03129719, 0.10938915]),
 'component_risk_contribution_pct': array([0.10426603, 0.03658624, 0.07488505, 0.11144182, 0.02478864,
        0.08495748, 0.045895  , 0.00458162, 0.02614798, 0.04891175,
        0.09917916, 0.03563909, 0.03713978, 0.01929903, 0.09037799,
        0.03468238, 0.12122096]),
 'component_total_return_contribution': l

In [95]:
full_stats_s = save_results(sharpe_results, rebalance_results_s, which='sharpe')
full_stats_s

{'return': 0.7686838703001058,
 'volatility': 0.8734450822860235,
 'sharpe_ratio': 0.8800597609276917,
 'diversification_ratio': 3.4530545367037964,
 'marginal_risk_contribution': array([0.11217468, 0.05259085, 0.07107542, 0.07605781, 0.02316675,
        0.06482448, 0.05178736, 0.00632779, 0.01190869, 0.03165231,
        0.05572841, 0.05446169, 0.02283965, 0.03433997, 0.04845464,
        0.03789806, 0.11815652]),
 'component_risk_contribution': array([0.11217468, 0.05259085, 0.07107542, 0.07605781, 0.02316675,
        0.06482448, 0.05178736, 0.00632779, 0.01190869, 0.03165231,
        0.05572841, 0.05446169, 0.02283965, 0.03433997, 0.04845464,
        0.03789806, 0.11815652]),
 'component_risk_contribution_pct': array([0.12842786, 0.06021083, 0.08137366, 0.08707796, 0.02652342,
        0.07421701, 0.05929092, 0.00724464, 0.01363416, 0.03623846,
        0.06380299, 0.06235273, 0.02614892, 0.03931555, 0.05547532,
        0.04338917, 0.13527641]),
 'component_total_return_contribution': l